In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [5]:
PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')

In [6]:
from pinecone import Pinecone,ServerlessSpec
pine = Pinecone(api_key=PINECONE_API_KEY)
# pine = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

- 인덱스 생성(서버리스)

In [7]:
index_name='wiki'

In [7]:
pine.create_index(
    name=index_name,
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(cloud='aws',region='us-east-1')
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-3h3vqd5.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

- 임베딩 객체 생성

In [83]:
# 사용할 인덱스 가져오기
index = pine.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

- 임베딩 모델 생성

In [ ]:
# 임베딩 모델의 차원(dimension) 과 생성한 인덱스의 차원(dimension) 이 같아야 합니다.
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

- 벡터DB 에 저장할 데이터셋 가져오기
    - 영어로 된 20231101 버전의 위키백과 600만개의 row 중 100개만 가져오기(❌ 사용 안함)
    - https://huggingface.co/datasets/wikimedia/wikipedia/tree/main/20231101.en 에서 다운받은 데이터 사용(✅ 사용)
    - 데이터의 크기가 크면 임베딩 시간이 오래 걸림

In [2]:
from datasets import load_dataset
# pip install datasets
# dataset = load_dataset('wikimedia/wikipedia','20231101.en',split='train[:100]')

c:\Users\C117\miniconda3\envs\llmEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = load_dataset('parquet',data_files=['train-03.parquet'])

In [22]:
print(type(dataset))    # dict
print(dataset)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 156289
    })
})


In [72]:
data = dataset['train'][:1000]
print('len(data):',len(data))                       # dict 형태라 갯수 4개(칼럼이 4개)
print('len(data[\'text\']):',len(data['text']))     # key 각각의 data 갯수 1000개씩


len(data): 4
len(data['text']): 1000


In [73]:
print(data.keys())

dict_keys(['id', 'url', 'title', 'text'])


In [76]:
data['text'][0]

'Niklas Hogner (born 29 September 1984 in Linköping, Sweden) is a Swedish figure skater. Until 2003, he competed as a singles skater, winning four Swedish junior national titles and competing at the World Junior Figure Skating Championships.\n\nHe switched to pair skating, teaming up with partner Angelika Pylkina in 2003. They were the first Swedish pairs team to compete internationally since 1962. They twice placed 5th at the World Junior Championships and won three bronze medals on the Junior Grand Prix circuit. They won the bronze medal at the 2006 Nebelhorn Trophy and won the Nordic Championships. They ended their partnership in 2007.\n\nPrograms \n(with Pylkina)\n\nResults\n\nPair skating with Pylkina\n\nSingle skating\n\nReferences\n\nExternal links\n\n \n \n\n1984 births\nLiving people\nSportspeople from Linköping\nSwedish male single skaters\nSwedish male pair skaters'

In [74]:
data[50] # 오류남

KeyError: 50

In [77]:
data = dataset['train'].select(range(100))
print('len(data) :',len(data))

len(data) : 100


In [78]:
data[50]    # 

{'id': '8125821',
 'url': 'https://en.wikipedia.org/wiki/List%20of%20programs%20broadcast%20by%20G4',
 'title': 'List of programs broadcast by G4',
 'text': "This is a list of television programs formerly broadcast by the U.S. cable television channel G4.\n\nSecond iteration\n\nFinal programming\n\nOriginal\n\n Xplay (2004–2013; 2013–2014 (reruns); 2021–2022)\n Attack of the Show! (2005–2013; 2021–2022)\n Invitation to Party (2021–2022)\n Ninja Warrior (Sasuke and Kunoichi) (2006–2012, 2021–2022 (reruns))\n Unbeatable Banzuke (2008–2010; 2021–2022 (reruns))\n G4 Vault (2021–2022)\n G4's Crash Course (2021–2022)\n G4 Specials (2002–2012; 2021–2022)\n Scott the Woz (2021–2022)\n Name Your Price (2022)\n G4 Gameday LCS (2022)\n Arena (2002–2005; 2022)\n Hey, Donna! (2022)\n God of Work (2022)\n\nAcquired\n Starcade (2002–2004; 2021-2022)\n Takeshi's Castle Thailand (2021-2022)\n Viva La Dirt League (2022)\n Smosh (2022)\n\nFormer programming\n\nOriginal\n Boosted (February–July 2021 (YouT

- 청크
    - splitter 객체를 생성해서 문자열 나누기

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [40]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,             # 텍스트 분할 크기
    chunk_overlap=20,           # 분할할 텍스트의 중첩 크기
    length_function=len,
    separators=['\n',' ']       # 분할할때 사용할 텍스트(단어) 구분자
)

- 청킹 후 임베딩 -> 업서트 : batch_size

In [ ]:
from uuid import uuid4
import time

# 한 번에 처리(업서트)할 데이터의 개수를 지정합니다.
# batch_size 만큼 청크(chunk)를 모은 후, 한 번에 임베딩과 업서트를 수행합니다.
batch_size = 30

# 텍스트 청크(chunk)들을 임시로 저장할 리스트입니다.
# 일정 개수(batch_size)만큼 모이면 임베딩 처리 후 벡터DB에 업서트됩니다.
texts = []

# 각 텍스트 청크에 대응하는 메타데이터(문서 ID, 제목, URL 등)를 저장할 리스트입니다.
# texts 리스트와 인덱스가 동일하게 매칭됩니다.
metas = []

# 지금까지 처리한 청크(chunk)의 총 개수를 세는 카운터입니다.
# batch_size 개수에 도달하면 임베딩 및 업서트 작업을 수행하고 다시 0부터 세기 시작합니다.
count = 0

# 데이터셋의 각 샘플에 대해 반복합니다.  tqdm 은 data를 감싸기만 하여 진행 상황을 표시하도록 합니다.
for i, sample in enumerate(data):
    full_text = sample['text']          # Wikipedia 문서 텍스트 -> 청킹 후 임베딩
    
    # metadata key 구성은 임의로 합니다. 청킹된 데이터의 소속을 구별
    metadata = {
        'wiki_id': sample["id"],        # Wikipedia 문서 ID
        'url':sample['url'],            # Wikipedia 문서 URL
        'title': sample["title"]        # Wikipedia 문서 제목
    }


    chunks = splitter.split_text(full_text)  # 텍스트를 청크로 분할합니다.
    print(len(chunks))

    # 각 청크에 대해 반복합니다.
    for i, chunk in enumerate(chunks):
        # 실제로 벡터BD 에 업서트할 record
        record = {
            'chunk_id': i,  # 청크 ID
            'head_chunk': chunk[:200],  # 전체 텍스트
            **metadata,  # 메타데이터 언패킹
        }

        texts.append(chunk)     # 청크를 텍스트 목록에 추가합니다.
        metas.append(record)    # 메타데이터를 메타데이터 목록에 추가합니다.

        count += 1  # 처리된 청크 수를 증가시킵니다.

        # batch_size만큼의 청크를 처리 : 임베딩 -> 업서트
        if count % batch_size == 0:
            # Pinecone 인덱스에 청크를 추가합니다.
            ids = [str(uuid4()) for _ in range(len(texts))]
            embeddings = embedding.embed_documents(texts)
            index.upsert(
                vectors=zip(ids, embeddings, metas),
                namespace="wiki-ns1")
            
            # 청크 목록과 메타데이터 목록을 비웁니다.
            texts = []
            metas = []

            # 1초 대기합니다.
            time.sleep(1)
